<a href="https://colab.research.google.com/github/Isometric1/ML/blob/main/KaggleSberPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
from math import *
! pip install catboost
from sklearn.model_selection import train_test_split
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error
from sklearn.metrics.pairwise import normalize
! pip install phik
import phik
from sklearn.metrics import log_loss
import phik

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! gdown --id 1Jftp2ZQBlMpwgNzkWPI8h6joyGY374aN

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Jftp2ZQBlMpwgNzkWPI8h6joyGY374aN
To: /content/sberbank-russian-housing-market.zip
100% 22.7M/22.7M [00:00<00:00, 94.3MB/s]


In [3]:
! unzip /content/sberbank-russian-housing-market.zip -d /content/sberbank-russian-housing-market

Archive:  /content/sberbank-russian-housing-market.zip
  inflating: /content/sberbank-russian-housing-market/data_dictionary.txt  
  inflating: /content/sberbank-russian-housing-market/macro.csv.zip  
  inflating: /content/sberbank-russian-housing-market/sample_submission.csv.zip  
  inflating: /content/sberbank-russian-housing-market/test.csv.zip  
  inflating: /content/sberbank-russian-housing-market/train.csv.zip  


In [4]:
! unzip  /content/sberbank-russian-housing-market/macro.csv.zip -d /content/sberbank-russian-housing-market/macro.csv
! unzip /content/sberbank-russian-housing-market/test.csv.zip -d /content/sberbank-russian-housing-market/test.csv
! unzip /content/sberbank-russian-housing-market/train.csv.zip -d /content/sberbank-russian-housing-market/train.csv

Archive:  /content/sberbank-russian-housing-market/macro.csv.zip
  inflating: /content/sberbank-russian-housing-market/macro.csv/macro.csv  
Archive:  /content/sberbank-russian-housing-market/test.csv.zip
  inflating: /content/sberbank-russian-housing-market/test.csv/test.csv  
   creating: /content/sberbank-russian-housing-market/test.csv/__MACOSX/
  inflating: /content/sberbank-russian-housing-market/test.csv/__MACOSX/._test.csv  
Archive:  /content/sberbank-russian-housing-market/train.csv.zip
  inflating: /content/sberbank-russian-housing-market/train.csv/train.csv  
   creating: /content/sberbank-russian-housing-market/train.csv/__MACOSX/
  inflating: /content/sberbank-russian-housing-market/train.csv/__MACOSX/._train.csv  


In [114]:

test = pd.read_csv('/content/sberbank-russian-housing-market/test.csv/test.csv')
train = pd.read_csv('/content/sberbank-russian-housing-market/train.csv/train.csv')
macro = pd.read_csv('/content/sberbank-russian-housing-market/macro.csv/macro.csv')

In [115]:
train['life_sq'] =  train['life_sq'].where(~((train['life_sq'].isna())), '0')
train['build_year'] =  train['build_year'].where(~((train['build_year'].isna())), '1980')
train['kitch_sq'] =  train['kitch_sq'].where(~((train['kitch_sq'].isna())), '0')
train['raion_popul'] =  train['raion_popul'].where(~((train['raion_popul'].isna())), '0')
train['green_zone_part'] =  train['green_zone_part'].where(~((train['green_zone_part'].isna())), '0')
train['children_preschool'] =  train['children_preschool'].where(~((train['children_preschool'].isna())), '0')
train['full_sq'] =  train['full_sq'].where(~((train['full_sq'].isna())), '0')

train['material'] =  train['material'].where(~((train['material'].isna())), '0')
train['num_room'] =  train['num_room'].where(~((train['num_room'].isna())), '0')


test['life_sq'] =  test['life_sq'].where(~((test['life_sq'].isna())), '0')
test['build_year'] =  test['build_year'].where(~((test['build_year'].isna())), '1980')
test['kitch_sq'] =  test['kitch_sq'].where(~((test['kitch_sq'].isna())), '0')
test['raion_popul'] =  test['raion_popul'].where(~((test['raion_popul'].isna())), '0')
test['green_zone_part'] =  test['green_zone_part'].where(~((test['green_zone_part'].isna())), '0')
test['children_preschool'] =  test['children_preschool'].where(~((test['children_preschool'].isna())), '0')
test['full_sq'] =  test['full_sq'].where(~((test['full_sq'].isna())), '0')

test['material'] =  test['material'].where(~((test['material'].isna())), '0')
test['num_room'] =  test['num_room'].where(~((test['num_room'].isna())), '0')




In [120]:
test.head()



,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,...,0,0,0,1,10,1,0,14,1,5.771091e+06
1,30475,2015-07-01,79.2,0,8,17,1,0.0,3,1.0,...,1,1,0,2,11,0,1,12,1,8.809451e+06
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,...,11,4,0,10,21,0,10,71,11,5.468948e+06
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,...,1,2,0,0,10,0,0,2,0,6.416198e+06
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,...,1,1,0,2,12,0,1,11,1,5.202944e+06


In [26]:
macro.head()

,timestamp,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_deflator,balance_trade,balance_trade_growth,usdrub,...,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,2010-01-01,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
1,2010-01-02,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
2,2010-01-03,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
3,2010-01-04,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.905,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
4,2010-01-05,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.836,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN


In [117]:
cat_features = []
X = ['full_sq','life_sq','build_year','num_room','kitch_sq','raion_popul','green_zone_part','children_preschool']
Y = ['price_doc']

In [94]:
train.columns

Index(['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor',
       'material', 'build_year', 'num_room', 'kitch_sq',
       ...
       'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000',
       'cafe_count_5000_price_high', 'big_church_count_5000',
       'church_count_5000', 'mosque_count_5000', 'leisure_count_5000',
       'sport_count_5000', 'market_count_5000', 'price_doc'],
      dtype='object', length=292)

In [95]:
test.columns

Index(['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor',
       'material', 'build_year', 'num_room', 'kitch_sq',
       ...
       'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500',
       'cafe_count_5000_price_4000', 'cafe_count_5000_price_high',
       'big_church_count_5000', 'church_count_5000', 'mosque_count_5000',
       'leisure_count_5000', 'sport_count_5000', 'market_count_5000'],
      dtype='object', length=291)

In [96]:
train.iloc[0:5, 0:20]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion
0,1,2011-08-20,43,27.0,4.0,NaN,0,1980,0,0,NaN,Investment,Bibirevo,6.407578e+06,155572,0.189727,0.000070,9576,5001.0,5
1,2,2011-08-23,34,19.0,3.0,NaN,0,1980,0,0,NaN,Investment,Nagatinskij Zaton,9.589337e+06,115352,0.372602,0.049637,6880,3119.0,5
2,3,2011-08-27,43,29.0,2.0,NaN,0,1980,0,0,NaN,Investment,Tekstil'shhiki,4.808270e+06,101708,0.112560,0.118537,5879,1463.0,4
3,4,2011-09-01,89,50.0,9.0,NaN,0,1980,0,0,NaN,Investment,Mitino,1.258354e+07,178473,0.194703,0.069753,13087,6839.0,9
4,5,2011-09-05,77,77.0,4.0,NaN,0,1980,0,0,NaN,Investment,Basmannoe,8.398461e+06,108171,0.015234,0.037316,5706,3240.0,7


In [118]:
model = CatBoostRegressor(cat_features = cat_features, learning_rate=0.2, random_state = 42, loss_function = 'RMSE',early_stopping_rounds= 100)

model.fit(train[X],train[Y])


0:	learn: 4341313.4683579	total: 5.16ms	remaining: 5.16s
1:	learn: 4029896.8483277	total: 9.52ms	remaining: 4.75s
2:	learn: 3800402.2939625	total: 13.3ms	remaining: 4.42s
3:	learn: 3637398.1352288	total: 17.1ms	remaining: 4.26s
4:	learn: 3501660.9106149	total: 21.4ms	remaining: 4.26s
5:	learn: 3411904.8480948	total: 26.1ms	remaining: 4.32s
6:	learn: 3346280.7685906	total: 30.2ms	remaining: 4.29s
7:	learn: 3291756.8738055	total: 34.3ms	remaining: 4.25s
8:	learn: 3245876.9168383	total: 38.6ms	remaining: 4.25s
9:	learn: 3203286.1123084	total: 42.2ms	remaining: 4.18s
10:	learn: 3172922.5136006	total: 46ms	remaining: 4.14s
11:	learn: 3146109.9667877	total: 49.9ms	remaining: 4.11s
12:	learn: 3118166.2622233	total: 53.8ms	remaining: 4.09s
13:	learn: 3095243.9206427	total: 57.7ms	remaining: 4.06s
14:	learn: 3080574.9765703	total: 61.3ms	remaining: 4.03s
15:	learn: 3056225.5838779	total: 65.4ms	remaining: 4.02s
16:	learn: 3029763.4912049	total: 69.1ms	remaining: 4s
17:	learn: 3015610.2861243	to

In [119]:
test['price_doc'] = model.predict(test[X])

In [121]:
test[['id','price_doc']]

,id,price_doc
0,30474,5.771091e+06
1,30475,8.809451e+06
2,30476,5.468948e+06
3,30477,6.416198e+06
4,30478,5.202944e+06
...,...,...
7657,38131,8.824801e+06
7658,38132,4.891164e+06
7659,38133,4.309455e+06
7660,38134,5.451064e+06


In [122]:
test[['id','price_doc']].to_csv('/content/Kaggle.csv')